In [2]:
import pandas as pd
import datetime

In [3]:
df = pd.read_csv('parsed2.csv',header=None, names = range(18), nrows = 11000000)

C:\Users\Hurle\AppData\Local\Temp\ipykernel_1644\1427644825.py:1: DtypeWarning: Columns (4,6,8,9,10,11,12,13,14,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('parsed2.csv',header=None, names = range(18), nrows = 11000000)


In [3]:
df[df[0] == "S"]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,S,0,0,10970642174571,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237169,S,0,0,14400000186600,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10411212,S,0,0,34200000075128,Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
sys_df = df[df[0] == "S"]
sys_df = sys_df[[0,1,2,3,4]]
sys_df.columns = ["Code","num1","num2","ts","sys_code"]

In [5]:
stock_df = df[df[0] == 'R']
stock_df = stock_df.reset_index(drop=True)
stock_df.columns = ["Message_type","Stock_locate","Tracking_num","Ts","Stock","Market_cat","Financial_status","Round_lot_size","Round_lots_only","issue_class","issue_subtype","authenticity","SSTI","IPO","LULD","ETP","Lev","Inv"]

In [6]:
stock_df

,Message_type,Stock_locate,Tracking_num,Ts,Stock,Market_cat,Financial_status,Round_lot_size,Round_lots_only,issue_class,issue_subtype,authenticity,SSTI,IPO,LULD,ETP,Lev,Inv
0,R,1,0,11240690615180,A,N,,100,N,C,Z,P,N,,1,N,0.0,N
1,R,2,0,11240690740789,AA,N,,100,N,C,Z,P,N,,1,N,1.0,N
2,R,3,0,11240690786821,AAAU,P,,100,N,Q,I,P,N,,2,Y,1.0,N
3,R,4,0,11240690840751,AABA,Q,N,100,N,C,Q,P,N,N,2,N,1.0,N
4,R,5,0,11240690884618,AAC,N,,100,N,C,Z,P,N,,2,N,0.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8844,R,8845,0,11240925383765,ZXYZ.A,Q,N,100,N,C,Z,T,N,N,2,N,1.0,N
8845,R,8846,0,11240925418549,ZXZZT,G,N,100,N,W,Z,T,N,N,2,N,1.0,N
8846,R,8847,0,11240925445584,ZYME,N,,100,N,C,Z,P,N,,2,N,1.0,N
8847,R,8848,0,11240925470514,ZYNE,G,N,100,N,C,Z,P,N,N,2,N,0.0,N


In [7]:
#TS operations
stock_df["Ts_seconds"] = stock_df['Ts']/1e9
stock_df["Ts_stamp"] = stock_df["Ts_seconds"].apply(lambda x: datetime.datetime(2023,2,27) + datetime.timedelta(seconds = x))

In [8]:
#Stock_name operations
stock_df["Stock_normalized"] = stock_df["Stock"].apply(lambda x: x.split(" ")[0])

In [9]:
#Create Dictionary of stock locate ids
locate_dic = dict(zip(stock_df["Stock_locate"],stock_df["Stock_normalized"]))
rev_locate = {v: k for k,v in locate_dic.items()}

In [10]:
def normalize_series_name(sers):
    return sers.apply(lambda x: x.split(" ")[0])

def normalize_time(sers):
    return sers.apply(lambda x: datetime.datetime(2023,2,27) + datetime.timedelta(seconds = x/1e9))


In [12]:
exec_df = df[df[0] == 'E'][[0,1,2,3,4,5,6]].reset_index(drop=True)
exec_df.columns = ["Message_code","Locate","Track","Ts","Ref","Shares","Match"]
exec_df["Stock_normalized"] = exec_df["Locate"].apply(lambda x: locate_dic[x].split(' ')[0])
exec_df["Ts_norm"] = normalize_time(exec_df[exec_df["Stock_normalized"] == 'AAPL']["Ts"])

In [13]:
aapl_df = exec_df[exec_df["Stock_normalized"] == 'AAPL'].reset_index(drop=True)

In [62]:
aapl_adds = df[(df[0]=='A') & (df[7] == 'AAPL    ')]
aapl_adds = aapl_adds.iloc[:,0:9]
aapl_adds = aapl_adds.reset_index(drop=True)
aapl_adds.columns = ["Mess","Stock","Track","Ts","Order","BS","Shares","Stock_name","Price"]
aapl_adds["Stock_name"] = aapl_adds["Stock_name"].apply(lambda x: x.split(" ")[0])
aapl_adds["Price"] = aapl_adds["Price"].astype(float)
aapl_adds["Price"] /= 1e4
aapl_adds["Order"] = aapl_adds["Order"].astype(int)

In [63]:
aapl_adds

,Mess,Stock,Track,Ts,Order,BS,Shares,Stock_name,Price
0,A,14,0,14400023941081,17613,B,53,AAPL,189.86
1,A,14,0,14400756946890,52437,S,1,AAPL,217.05
2,A,14,0,14400761460097,52669,B,1,AAPL,205.20
3,A,14,0,14400769923694,53005,B,3,AAPL,201.48
4,A,14,0,14400771392054,53069,S,2,AAPL,228.00
...,...,...,...,...,...,...,...,...,...
5571,A,14,0,34201097623576,9919517,B,100.0,AAPL,208.61
5572,A,14,0,34201097676372,9919537,B,5.0,AAPL,208.61
5573,A,14,0,34201140584010,9927785,S,1200.0,AAPL,208.66
5574,A,14,0,34201259074296,9948741,B,250.0,AAPL,208.52


In [64]:
aapl_mpid_adds = df[(df[0]=='F') & (df[7] == 'AAPL    ')]
aapl_mpid_adds = aapl_mpid_adds.iloc[:,0:10]
aapl_mpid_adds = aapl_mpid_adds.reset_index(drop=True)
aapl_mpid_adds.columns = ["Mess","Stock","track","ts","match","side","shares","stock_name","price","mpid"]
aapl_mpid_adds["stock_name"] = aapl_mpid_adds["stock_name"].apply(lambda x: x.split(" ")[0])
aapl_mpid_adds["price"] = aapl_mpid_adds["price"].astype(float)
aapl_mpid_adds["price"] /= 1e4
aapl_mpid_adds["match"] = aapl_mpid_adds["match"].astype(int)


In [25]:
aapl_mpid_adds

,Mess,Stock,track,ts,match,side,shares,stock_name,price,mpid
0,F,14,0,27537701776068,4009485,B,100.0,AAPL,0.01,NITE
1,F,14,0,27537701781491,4009489,S,100.0,AAPL,199999.99,NITE
2,F,14,0,32047771123323,6600101,B,100.0,AAPL,171.47,SGAS
3,F,14,0,32047771140623,6600105,S,100.0,AAPL,244.24,SGAS
4,F,14,0,32050751925530,6606301,B,100.0,AAPL,171.47,SGAS
...,...,...,...,...,...,...,...,...,...,...
89,F,14,0,34200284491468,5510453,S,40,AAPL,209.00,UBSS
90,F,14,0,34200284491468,7325717,S,62,AAPL,209.00,UBSS
91,F,14,0,34200703980871,6018497,S,30.0,AAPL,209.20,UBSS
92,F,14,0,34200774686779,9789945,B,100.0,AAPL,166.85,UBSS


In [65]:
aapl_cancels = df[(df[0]=='X') & (df[1] == 14)]
aapl_cancels = aapl_cancels.iloc[:,0:6].reset_index(drop=True)
aapl_cancels.columns = ["Mess","stock","track","ts","match","shares"]
aapl_cancels["stock_name"] = aapl_cancels["stock"].apply(lambda x: locate_dic[x].split(" ")[0])
aapl_cancels["match"] = aapl_cancels["match"].astype(int)

In [35]:
aapl_cancels

,Mess,stock,track,ts,match,shares,stock_name
2065877,X,14,0,19843248546060,665097,4,AAPL
5120210,X,14,0,29029436117009,5032457,100,AAPL
5187733,X,14,0,29163339026316,5136353,5,AAPL
5352189,X,14,0,29585349397538,5287781,574,AAPL
5385626,X,14,0,29670384095385,5287781,10,AAPL
7005409,X,14,0,32343722414878,6731421,6,AAPL
7007777,X,14,0,32344313096913,6731421,1,AAPL
7016355,X,14,0,32346379562677,6731421,7,AAPL
7017628,X,14,0,32346952246952,6731421,32,AAPL
7018794,X,14,0,32347073361379,6731421,1,AAPL


In [67]:
aapl_dels = df[(df[0]=='D') & (df[1] == 14)]
aapl_dels = aapl_dels.iloc[:,0:5].reset_index(drop=True)
aapl_dels.columns = ["Mess","stock","track","ts","match"]
aapl_dels["match"] = aapl_dels["match"].astype(int)


In [66]:
aapl_dels["match"]

0         99549
1         99553
2         99645
3        104297
4        104177
         ...   
3218    9834853
3219    9834701
3220    9844553
3221    9919517
3222    9844505
Name: match, Length: 3223, dtype: object

In [68]:
aapl_replaces = df[(df[0]=='U') & (df[1] == 14)]
aapl_replaces = aapl_replaces.iloc[:,0:8].reset_index(drop=True)
aapl_replaces.columns = ["Mess","stock","track","ts","match1","match2","shares","price"]
aapl_replaces["price"] = aapl_replaces["price"].astype(float)/1e4
aapl_replaces["match1"] = aapl_replaces["match1"].astype(int)
aapl_replaces["match2"] = aapl_replaces["match2"].astype(int)

In [69]:
aapl_replaces

,Mess,stock,track,ts,match1,match2,shares,price
0,U,14,0,34140265906478,9170025,9173997,100,214.73
1,U,14,0,34156614968284,9170021,9210005,100,202.07
2,U,14,0,34156615015926,9173997,9210013,100,214.71
3,U,14,0,34156630598645,9210013,9210041,100,214.66
4,U,14,0,34164035937297,9210005,9225325,100,202.00
...,...,...,...,...,...,...,...,...
63,U,14,0,34200931996535,9834729,9850401,100.0,214.97
64,U,14,0,34201097668069,9834953,9919529,16.0,208.61
65,U,14,0,34201097794810,9837441,9919569,100.0,202.35
66,U,14,0,34201143374704,9919569,9928173,100.0,202.25


In [103]:
price_dic = dict(zip(aapl_adds["Order"],aapl_adds["Price"]))
price_dic.update(zip(aapl_mpid_adds["match"],aapl_mpid_adds["price"]))
price_dic.update(zip(aapl_replaces["match2"],aapl_replaces["price"]))

In [85]:
price_dic

{17613: 189.86,
 52437: 217.05,
 52669: 205.2,
 53005: 201.48,
 53069: 228.0,
 53713: 190.0,
 53761: 207.22,
 54141: 175.09,
 54689: 187.5,
 55197: 185.0,
 55373: 214.0,
 55465: 240.0,
 55925: 211.0,
 57077: 190.01,
 57097: 204.53,
 57465: 219.6,
 57641: 178.9,
 57897: 195.0,
 57993: 209.9,
 99549: 207.23,
 99553: 207.18,
 99645: 209.92,
 101737: 209.0,
 101757: 200.0,
 101761: 209.01,
 101765: 208.96,
 102093: 255.0,
 102109: 210.6,
 102125: 215.0,
 102409: 182.0,
 102601: 225.0,
 102609: 217.5,
 102645: 250.0,
 102653: 171.0,
 102757: 196.83,
 102793: 169.5,
 104177: 209.91,
 104297: 209.9,
 115153: 208.0,
 128393: 209.68,
 128445: 209.67,
 128449: 209.72,
 135173: 209.3,
 135213: 209.2,
 138593: 209.8,
 139645: 209.35,
 139805: 209.21,
 139829: 210.2,
 140421: 209.23,
 140473: 209.4,
 140789: 209.2,
 141005: 209.25,
 142589: 209.2,
 142977: 209.23,
 145785: 209.24,
 146845: 209.45,
 148381: 209.25,
 148597: 209.26,
 151809: 209.28,
 152065: 209.19,
 153065: 209.14,
 153181: 209.17,


In [104]:
aapl_df["Price"] = aapl_df["Ref"].apply(lambda x: price_dic[int(x)])

In [111]:
aapl_df

,Message_code,Locate,Track,Ts,Ref,Shares,Match,Stock_normalized,Ts_norm,Price
0,E,14,2,14602436281050,146845,100,17811.0,AAPL,2023-02-27 04:03:22.436281,209.45
1,E,14,2,14912188415020,140473,8,17905.0,AAPL,2023-02-27 04:08:32.188415,209.40
2,E,14,2,14914352456248,140473,92,17906.0,AAPL,2023-02-27 04:08:34.352456,209.40
3,E,14,4,14914352456248,220569,8,17907.0,AAPL,2023-02-27 04:08:34.352456,209.34
4,E,14,2,14936714684440,242225,300,17915.0,AAPL,2023-02-27 04:08:56.714684,209.45
...,...,...,...,...,...,...,...,...,...,...
642,E,14,2,34200892798462,9834701,1,177231.0,AAPL,2023-02-27 09:30:00.892798,208.51
643,E,14,2,34200945876852,9834725,100,187597.0,AAPL,2023-02-27 09:30:00.945877,208.51
644,E,14,4,34200945876852,9834953,16,187598.0,AAPL,2023-02-27 09:30:00.945877,208.51
645,E,14,2,34201188675197,9919529,16,222597.0,AAPL,2023-02-27 09:30:01.188675,208.61


In [110]:
#VWAP Calc:
sum(aapl_df["Shares"].astype(int) * aapl_df["Price"])/aapl_df["Shares"].astype(int).sum()

208.3076886774643

In [54]:
aapl_mpid_adds

,Mess,Stock,track,ts,match,side,shares,stock_name,price,mpid
0,F,14,0,27537701776068,4009485,B,100.0,AAPL,0.01,NITE
1,F,14,0,27537701781491,4009489,S,100.0,AAPL,199999.99,NITE
2,F,14,0,32047771123323,6600101,B,100.0,AAPL,171.47,SGAS
3,F,14,0,32047771140623,6600105,S,100.0,AAPL,244.24,SGAS
4,F,14,0,32050751925530,6606301,B,100.0,AAPL,171.47,SGAS
...,...,...,...,...,...,...,...,...,...,...
89,F,14,0,34200284491468,5510453,S,40,AAPL,209.00,UBSS
90,F,14,0,34200284491468,7325717,S,62,AAPL,209.00,UBSS
91,F,14,0,34200703980871,6018497,S,30.0,AAPL,209.20,UBSS
92,F,14,0,34200774686779,9789945,B,100.0,AAPL,166.85,UBSS


In [59]:
stock_df["Ts_seconds"] = datetime.datetime(2023,2,27) + datetime.timedelta(seconds = 11240)

0       11240.690615
1       11240.690741
2       11240.690787
3       11240.690841
4       11240.690885
            ...     
8844    11240.925384
8845    11240.925419
8846    11240.925446
8847    11240.925471
8848    11240.925496
Name: Ts_seconds, Length: 8849, dtype: float64

In [62]:
aapl_adds
aapl_cancels
aapl_mpid_adds
aapl_dels
aapl_replaces

In [112]:
aapl_adds

,Mess,Stock,Track,Ts,Order,BS,Shares,Stock_name,Price
0,A,14,0,14400023941081,17613,B,53,AAPL,189.86
1,A,14,0,14400756946890,52437,S,1,AAPL,217.05
2,A,14,0,14400761460097,52669,B,1,AAPL,205.20
3,A,14,0,14400769923694,53005,B,3,AAPL,201.48
4,A,14,0,14400771392054,53069,S,2,AAPL,228.00
...,...,...,...,...,...,...,...,...,...
5571,A,14,0,34201097623576,9919517,B,100.0,AAPL,208.61
5572,A,14,0,34201097676372,9919537,B,5.0,AAPL,208.61
5573,A,14,0,34201140584010,9927785,S,1200.0,AAPL,208.66
5574,A,14,0,34201259074296,9948741,B,250.0,AAPL,208.52


In [113]:
aapl_mpid_adds

,Mess,Stock,track,ts,match,side,shares,stock_name,price,mpid
0,F,14,0,27537701776068,4009485,B,100.0,AAPL,0.01,NITE
1,F,14,0,27537701781491,4009489,S,100.0,AAPL,199999.99,NITE
2,F,14,0,32047771123323,6600101,B,100.0,AAPL,171.47,SGAS
3,F,14,0,32047771140623,6600105,S,100.0,AAPL,244.24,SGAS
4,F,14,0,32050751925530,6606301,B,100.0,AAPL,171.47,SGAS
...,...,...,...,...,...,...,...,...,...,...
89,F,14,0,34200284491468,5510453,S,40,AAPL,209.00,UBSS
90,F,14,0,34200284491468,7325717,S,62,AAPL,209.00,UBSS
91,F,14,0,34200703980871,6018497,S,30.0,AAPL,209.20,UBSS
92,F,14,0,34200774686779,9789945,B,100.0,AAPL,166.85,UBSS


In [118]:
aapl_adds.columns = ['Mess', 'Stock', 'track', 'ts', 'match', 'side', 'shares', 'stock_name','price']

In [131]:
adds_df = pd.concat([aapl_adds,aapl_mpid_adds])

In [125]:
aapl_dels

,Mess,stock,track,ts,match
0,D,14,0,14409720576443,99549
1,D,14,0,14409720652458,99553
2,D,14,0,14417030024911,99645
3,D,14,0,14531727649781,104297
4,D,14,0,14531727713641,104177
...,...,...,...,...,...
3218,D,14,0,34200914121450,9834853
3219,D,14,0,34200914121667,9834701
3220,D,14,0,34201097673258,9844553
3221,D,14,0,34201101642862,9919517


In [126]:
aapl_replaces

,Mess,stock,track,ts,match1,match2,shares,price
0,U,14,0,34140265906478,9170025,9173997,100,214.73
1,U,14,0,34156614968284,9170021,9210005,100,202.07
2,U,14,0,34156615015926,9173997,9210013,100,214.71
3,U,14,0,34156630598645,9210013,9210041,100,214.66
4,U,14,0,34164035937297,9210005,9225325,100,202.00
...,...,...,...,...,...,...,...,...
63,U,14,0,34200931996535,9834729,9850401,100.0,214.97
64,U,14,0,34201097668069,9834953,9919529,16.0,208.61
65,U,14,0,34201097794810,9837441,9919569,100.0,202.35
66,U,14,0,34201143374704,9919569,9928173,100.0,202.25


In [124]:
aapl_cancels

,Mess,stock,track,ts,match,shares,stock_name
0,X,14,0,19843248546060,665097,4,AAPL
1,X,14,0,29029436117009,5032457,100,AAPL
2,X,14,0,29163339026316,5136353,5,AAPL
3,X,14,0,29585349397538,5287781,574,AAPL
4,X,14,0,29670384095385,5287781,10,AAPL
5,X,14,0,32343722414878,6731421,6,AAPL
6,X,14,0,32344313096913,6731421,1,AAPL
7,X,14,0,32346379562677,6731421,7,AAPL
8,X,14,0,32346952246952,6731421,32,AAPL
9,X,14,0,32347073361379,6731421,1,AAPL


In [129]:
changes_df = pd.concat([aapl_replaces,aapl_cancels,aapl_dels]).sort_values(['ts'])

In [146]:
aapl_dels["match"][0]

99549

In [147]:
set(aapl_replaces["match2"]).intersection(set(aapl_dels["match"]))

{9383309}

In [153]:
adds_df

,Mess,Stock,track,ts,match,side,shares,stock_name,price,mpid
0,A,14,0,14400023941081,17613,B,53,AAPL,189.86,NaN
1,A,14,0,14400756946890,52437,S,1,AAPL,217.05,NaN
2,A,14,0,14400761460097,52669,B,1,AAPL,205.20,NaN
3,A,14,0,14400769923694,53005,B,3,AAPL,201.48,NaN
4,A,14,0,14400771392054,53069,S,2,AAPL,228.00,NaN
...,...,...,...,...,...,...,...,...,...,...
89,F,14,0,34200284491468,5510453,S,40,AAPL,209.00,UBSS
90,F,14,0,34200284491468,7325717,S,62,AAPL,209.00,UBSS
91,F,14,0,34200703980871,6018497,S,30.0,AAPL,209.20,UBSS
92,F,14,0,34200774686779,9789945,B,100.0,AAPL,166.85,UBSS


In [155]:
final_adds = adds_df[~adds_df["match"].isin(aapl_dels["match"])]

In [163]:
aapl_df

,Message_code,Locate,Track,Ts,Ref,Shares,Match,Stock_normalized,Ts_norm,Price
0,E,14,2,14602436281050,146845,100,17811.0,AAPL,2023-02-27 04:03:22.436281,209.45
1,E,14,2,14912188415020,140473,8,17905.0,AAPL,2023-02-27 04:08:32.188415,209.40
2,E,14,2,14914352456248,140473,92,17906.0,AAPL,2023-02-27 04:08:34.352456,209.40
3,E,14,4,14914352456248,220569,8,17907.0,AAPL,2023-02-27 04:08:34.352456,209.34
4,E,14,2,14936714684440,242225,300,17915.0,AAPL,2023-02-27 04:08:56.714684,209.45
...,...,...,...,...,...,...,...,...,...,...
642,E,14,2,34200892798462,9834701,1,177231.0,AAPL,2023-02-27 09:30:00.892798,208.51
643,E,14,2,34200945876852,9834725,100,187597.0,AAPL,2023-02-27 09:30:00.945877,208.51
644,E,14,4,34200945876852,9834953,16,187598.0,AAPL,2023-02-27 09:30:00.945877,208.51
645,E,14,2,34201188675197,9919529,16,222597.0,AAPL,2023-02-27 09:30:01.188675,208.61


In [166]:
buys = adds_df[adds_df["side"] == "B"]
sells = adds_df[adds_df["side"] == "S"]

In [171]:
just_needed = buys[["price","shares"]].sort_values("price", ascending=False)

In [188]:
just_needed.groupby(by=["price"])["shares"].sum().reset_index()

,price,shares
0,0.0001,201.0
1,0.0010,1.0
2,0.0100,111.0
3,0.0500,4000.0
4,0.1000,11.0
...,...,...
533,209.4500,100.0
534,209.4900,170.0
535,209.5000,300.0
536,209.5200,100.0
